# External data

There are two types, data provide at instantiation and data provided after instantiation.

Data provided at instantiation will be provided to the init through kwargs, will have to be derived from yaml code, so it should be short and simple, lists are going to be enough

Runtime provided data are those provided after instantiation, these could be voting records and are not restricted

## Instantiation
Yaml example for a plutrinominale district:
```
name: PIEMONTE 2 - 01
uninominali: 
    - 05 - ALESSANDRIA
    - 06 - ASTI
    - 07 - CUNEO
    - 08 - ALBA
seggi: 9
```
`name` is a standard parameter which will be used to find the specific instance, it's important it's always the same and unique, at least between members of the same Class

The configuration file to receive the above could be of the form:
```
uninominali:
  source: instantiation
  type: list
  class: Uninominale
seggi:
  name: num_seggi
  source: instantiation
  type: scalar
  class: int
```

In [ ]:
def gen_init(conf):
    Da testare !!!
    def __init__(self, **kwargs):
        for i in conf:
            n = conf[i].get('name', i)
            setattr(self, n, kwargs[i])
            if 'class' in conf[i]:
                self.types[i] = eval(conf[i]['class'])
                
    return __init__


def gen_accessors(conf):
    conf = copy.deepcopy(conf)
    lis_f = conf.keys()
    lis_f = map(lambda k: conf[k].get('name', k), lis_f)
    def gen_fun(var):
        def acc(self):
            return self.var
        return (var, acc)
    m = map(gen_fun, lis_f)
    fs = {}
    for i, f in m:
        fs[f'get_{i}'] = f
    return fs



## Runtime
These define a function which accepts the given data. Since it most likely will be a Dataframe I embed some data about the dataframe

In addition to the process as for instantiation-time argument passing I will add a few considerations to deal specifically with DataFrame like data, in particular I want to give the option of deriving from a single function multiple dataframes

Example configuration:
```
voti_sezione:
  targets:
    - name: candidato
      columns:
        - Candidato
        - VotiCand -> Voti
    - name: lista_cand
      columns:
        - Lista
        - Candidato
        - VotiLista -> Voti
```
Here all other columns will be discarded.

If target isn't present I'll just look for name and columns, if columns isn't present I won't do anything

In [ ]:
import pandas as pd
import copy

In [ ]:
import sys
import os

In [ ]:
import sys
sys.path.insert(1, '/home/lruffati/Documents/0.University/0.SistemiElettorali/')
import src

from src.Metaclasses import *
import src.GlobalVars as GlobalVars

In [3]:
class HubClass:
    def get_instance(self, *a, **kw):
        print(a, kw)
        return a[1]
    
GlobalVars.Hub = HubClass()

In [4]:
class cleanup(type):
    def __new__(mcs, *args, **kwargs):
        print(args, kwargs)
        return super().__new__(mcs, *args)

In [5]:
mc = type("meta_comb",(external,cleanup), {})
conf = {
    'prova': {'init':True, 'type': 'ClasseHub'},
 'provaNuova': {'name': 'provaConNome', 'type':'int'},
 'voti_sezione': { 'targets': [{'columns': ['c -> Candidato', 'vc -> Voti'],
    'name': 'candidato'},
   {'columns': ['l -> Lista', 'c', 'vl -> Voti'], 'name': 'lista_cand'}]}}
Classe = mc("Classe", (), {}, external=conf)

external processed:  {'prova': {'init': True, 'type': 'ClasseHub'}, 'provaNuova': {'name': 'provaConNome', 'type': 'int'}, 'voti_sezione': {'targets': [{'columns': ['c -> Candidato', 'vc -> Voti'], 'name': 'candidato'}, {'columns': ['l -> Lista', 'c', 'vl -> Voti'], 'name': 'lista_cand'}]}}
{'get_prova': <function external.gen_accessors.<locals>.gen_fun.<locals>.acc at 0x7fd33205a700>, 'get_provaConNome': <function external.gen_accessors.<locals>.gen_fun.<locals>.acc at 0x7fd33205a790>, 'get_candidato': <function external.gen_accessors.<locals>.gen_fun.<locals>.acc at 0x7fd33205a820>, 'get_lista_cand': <function external.gen_accessors.<locals>.gen_fun.<locals>.acc at 0x7fd33205a8b0>} {'give_prova': <function external.gen_providers.<locals>.gen_fun.<locals>.give at 0x7fd33205a940>, 'give_provaNuova': <function external.gen_providers.<locals>.gen_fun.<locals>.give at 0x7fd33205a9d0>, 'give_voti_sezione': <function external.gen_providers.<locals>.gen_fun.<locals>.give at 0x7fd33205aa60>}


In [6]:
inst_work = Classe(prova="Ciao")

In [7]:
inst_work.get_prova()

['Hub', 'HubClass', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']
('ClasseHub', 'Ciao') {}


'Ciao'

In [ ]:
import pandas as pd
df = pd.DataFrame({'c':['a','b','c','c','c'],'l':['a','b','c','d','e'],'vl':[10,20,5,7,8],'vc':[11,20,24,24,24]})
df

In [ ]:
inst_work.give_voti_sezione(df)

In [ ]:
inst_work.get_lista_cand()

In [ ]:
inst_work.give_provaNuova("123")


In [ ]:
inst_work.get_provaConNome()+1